In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import numpy as np
import pickle
import os


In [2]:
NODE = "01"
dataset_file = f"dataset/iot23_{NODE}.csv"
model_file = f"models/iot23_{NODE}"


In [3]:
df = pd.read_csv(dataset_file, nrows=400000, index_col=[0])

df

,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,...,missed_bytes,orig_bytes,orig_ip_bytes,orig_pkts,proto_icmp,proto_tcp,proto_udp,resp_bytes,resp_ip_bytes,resp_pkts
0,0,0,0,0,0,0,1,0,0,0,...,0,0,60,1,0,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,60,1,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,4344,157,2381,41,0,1,0,54305,56393,40
3,0,0,0,0,0,0,0,0,0,0,...,10136,90,2302,41,0,1,0,54305,56393,40
4,0,0,0,0,0,0,1,0,0,0,...,0,0,180,3,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,0,0,0,0,0,0,1,0,0,0,...,0,0,40,1,0,1,0,0,0,0
99995,0,0,0,0,0,0,1,0,0,0,...,0,0,40,1,0,1,0,0,0,0
99996,0,0,0,0,0,0,1,0,0,0,...,0,0,40,1,0,1,0,0,0,0
99997,0,0,0,0,0,0,1,0,0,0,...,0,0,40,1,0,1,0,0,0,0


In [4]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['label']


scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [5]:
if (os.path.isfile(model_file)):
    SVM_classifier = pickle.load(open(model_file, 'rb'))
else:
    SVM_classifier = SVC(C=1.0, cache_size=1500, verbose=False, max_iter=1000)
    
SVM_classifier.fit(X_train, Y_train)
print(SVM_classifier.score(X_test, Y_test))

/home/raghav/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


0.57225


In [38]:
pickle.dump(SVM_classifier, open(model_file, 'wb'))

In [39]:
print("Classifiction Report")

y_pred = SVM_classifier.predict(X_test)
print(y_pred)
print(classification_report(Y_test, y_pred))

Classifiction Report
['PartOfAHorizontalPortScan' 'PartOfAHorizontalPortScan'
 'PartOfAHorizontalPortScan' ... 'PartOfAHorizontalPortScan'
 'PartOfAHorizontalPortScan' 'PartOfAHorizontalPortScan']


/home/raghav/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                           precision    recall  f1-score   support

                   Benign       0.00      0.00      0.00      5959
                      C&C       1.00      0.29      0.44         7
         C&C-FileDownload       0.38      1.00      0.55         3
                    Okiru       0.00      0.00      0.00      2590
PartOfAHorizontalPortScan       0.57      1.00      0.73     11441

                 accuracy                           0.57     20000
                macro avg       0.39      0.46      0.34     20000
             weighted avg       0.33      0.57      0.42     20000



/home/raghav/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/raghav/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
